# Brief sketch of Julia's secret sauce

Julia's *secret sauce*

- **Ahead-of-time compilation**
- **Multiple dispatch**
- **Abstract types**

The following is not needed if running from the REPL:

In [1]:
using Pkg
Pkg.activate(temp=true)
Pkg.add("InteractiveUtils")
using InteractiveUtils

  Activating new environment at `/var/folders/4n/gvbmlhdc8xj973001s6vdyw00000gq/T/jl_ioz4EB/Project.toml`
   Resolving package versions...
    Updating `/private/var/folders/4n/gvbmlhdc8xj973001s6vdyw00000gq/T/jl_ioz4EB/Project.toml`
  [b77e0a4c] + InteractiveUtils
    Updating `/private/var/folders/4n/gvbmlhdc8xj973001s6vdyw00000gq/T/jl_ioz4EB/Manifest.toml`
  [2a0f44e3] + Base64
  [b77e0a4c] + InteractiveUtils
  [d6f4376e] + Markdown


## Just-in-time compilation

Here's how we define a new function in Julia:

In [2]:
add(x, y) = x + y

add (generic function with 1 method)

Let's see how long it takes to add two numbers:

In [3]:
t = time(); add(3, 5); time() - t

0.006323099136352539

Slow!! Why? Because Julia is a *compiled* language and does not
compile new code until it knows the type of arguments you want to
use.

Let's try again *with the same type* of argument:

In [4]:
t = time(); add(4, 7); time() - t

0.0002110004425048828

Fast!!! Why? Because Julia caches the compiled code and the types
are the same.

Let's try complex numbers:

In [5]:
t = time(); add(1 + 2im, 4 + 3im); time() - t

0.007798910140991211

Slow :-(

In [6]:
t = time(); add(3 + 6im, 7 - 5im); time() - t

0.0003070831298828125

Fast :-)

## Multiple dispatch

In [7]:
y = [1 2; 3 4]

2×2 Matrix{Int64}:
 1  2
 3  4

In [8]:
typeof(y)

Matrix{Int64} (alias for Array{Int64, 2})

Julia doesn't know how to apply `+` to a scalar and a
matrix. Uncomment the following line to see the error thrown:

In [9]:
# add(4, y)

So we *add* a more specialized version of our function (called a
*method*) to handle this case:

In [10]:
add(x::Int, y::Array{Int,2}) = x .+ y

add (generic function with 2 methods)

Here we are using the built-in broadcasted version of `+` which adds
the scalar `x` to each element of `y`. Now this works:

In [11]:
add(4, y)

2×2 Matrix{Int64}:
 5  6
 7  8

This is essentially what multiple dispatch is about. We use *all*
the arguments of a function to determine what specific method to
call. In a traditional object oriented language methods are owned by
objects (data structures) and we see syntax like `x.add(y)` which is
*single* dispatch on `x`.

In Julia *functions*, not objects, own *methods*:

In [12]:
methods(add)

# 2 methods for generic function "add":
[1] add(x::Int64, y::Matrix{Int64}) in Main.##258 at string:1
[2] add(x, y) in Main.##258 at string:1

Or, stated differently, there is less conflation of *structure* and
*behaviour* in Julia!

But, we're not out of the woods yet. Uncomment to see a new error
thrown:

In [13]:
# add(4.0, y)

Oh dear. Do we need to write a special method for every kind of
scalar and matrix???!

No, because abstract types come to the rescue...

## Abstract types

Everything in Julia has a type:

In [14]:
typeof(1 + 2im)

Complex{Int64}

In [15]:
typeof(rand(2,3))

Matrix{Float64} (alias for Array{Float64, 2})

These are examples of *concrete* types. But concrete types are
*organized* through the use of *abstract types*, in a type
heirarchy:

In [16]:
supertype(Int)

Signed

In [17]:
supertype(Signed)

Integer

In [18]:
supertype(Integer)

Real

In [19]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

In [20]:
4 isa Real

true

In [21]:
Bool <: Integer

true

In [22]:
String <: Integer

false

Now we can solve our problem: How to extend our `add` function to
arbitrary scalars and matrices:

In [23]:
add(x::Real, y::Matrix) = x .+ y

add (generic function with 3 methods)

In [24]:
add(4.0, rand(Bool, 2, 3))

2×3 Matrix{Float64}:
 5.0  5.0  4.0
 5.0  4.0  4.0

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*